In [1]:
import os
import sys
import re
import pickle
import numpy as np
from pprint import pprint
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras import backend
from tensorflow.python.client import device_lib
from tensorflow.keras.models import model_from_json
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
###### comment out on server#######
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
nltk.download('punkt')
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
#### Uncomment on server
# device = device_lib.list_local_devices()
# print(device)
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# backend.set_session(sess)

[nltk_data] Downloading package stopwords to /Users/Joe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Joe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#CONSTANTS
MAX_SONG_LENGTH = 2500
# MAX_NUM_WORDS = 20000
VALIDATION_SPLIT = 0.2
TEST_SPLIT = 0.2
SONG_PER_GENRE = 4500
learning_rate = .001
max_grad_norm = 1.
dropout = 0.5
EMBEDDING_DIM = 200

In [28]:
# PATH CONSTANTS
PICKLE_ROOT = 'data/lyrics/'
CHRISTIAN_PATH = 'Christian.pickle'
POP_PATH = 'Pop.pickle'
ROCK_PATH = 'Rock.pickle'
COUNTRY_PATH = 'Country.pickle'
RAP_PATH = 'Rap.pickle'

LYRIC_PATHS = [CHRISTIAN_PATH,POP_PATH,ROCK_PATH,COUNTRY_PATH,RAP_PATH]

EMBEDDING_PATH = 'data/glove_embeddings/'
EMBEDDING_FILE = 'glove.6B.'+str(EMBEDDING_DIM)+'d.txt'

MODEL_SAVE_FILE = 'cnn_model_1.0.json'
MODEL_SAVE_WEIGHTS_FILE = 'cnn_model_1.0.h5'

In [4]:
# Embedding
# Elmo could improve the word embeddings - need more research
# elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
print('loading embedding')
if not os.path.exists(EMBEDDING_PATH+EMBEDDING_FILE):
    print('Embeddings not found, downloading now')
    print(os.system('pwd'))
    os.system(' cd ' + DATA_PATH)
    os.system(' mkdir ' + EMBEDDING_DIR)
    os.system(' cd ' + EMBEDDING_DIR)
    os.system(' wget http://nlp.stanford.edu/data/glove.6B.zip')
    os.system(' unzip glove.6B.zip')
    os.system(' cd ../..')

glove_embeddings = {}
with open(EMBEDDING_PATH+EMBEDDING_FILE, encoding='utf-8') as emb_f:
    for line in emb_f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        glove_embeddings[word] = vector
print('finished loading embedding')


loading embedding
finished loading embedding


In [5]:
# Pickle extraction
# pickle looks like -> pickle_lyrics['lyrics'][('song_title', 'artist')]['lyrics']
# or - > pickle_lyrics['genre']
print('loading pickles')

pickle_lyrics = []
genre_index = {}
for i,l_path in enumerate(LYRIC_PATHS):
    if not os.path.exists(PICKLE_ROOT+l_path):
        print('problem occured looking for %s' %(PICKLE_ROOT+l_path))
        sys.exit()
    print(os.getcwd()+PICKLE_ROOT+l_path)
    loaded_lyrics = pickle.load(open(PICKLE_ROOT+l_path, "rb" ))
    genre_index[loaded_lyrics['genre']] = i
    pickle_lyrics.append(loaded_lyrics)
    print(len(loaded_lyrics['lyrics']))

print(len(pickle_lyrics))
print(genre_index)
print('finished loading pickles')



loading pickles
/Users/Joe/Applications/OneDrive/School/Spring 2019 - senior/NLP/project/tag_my_lyricsdata/lyrics/Christian.pickle
6895
/Users/Joe/Applications/OneDrive/School/Spring 2019 - senior/NLP/project/tag_my_lyricsdata/lyrics/Pop.pickle
5818
/Users/Joe/Applications/OneDrive/School/Spring 2019 - senior/NLP/project/tag_my_lyricsdata/lyrics/Rock.pickle
5596
/Users/Joe/Applications/OneDrive/School/Spring 2019 - senior/NLP/project/tag_my_lyricsdata/lyrics/Country.pickle
5368
/Users/Joe/Applications/OneDrive/School/Spring 2019 - senior/NLP/project/tag_my_lyricsdata/lyrics/Rap.pickle
4747
5
{'Christian': 0, 'Pop': 1, 'Rock': 2, 'Country': 3, 'Rap': 4}
finished loading pickles


In [6]:
def remove_stop_and_punct(article):
    final_article = []
    word_tokens = tokenizer.tokenize(article)
    filtered_article = [w for w in word_tokens if not w in stop_words]
    filtered_article = [x.lower() for x in filtered_article]
    filtered_song = ' '.join(filtered_article)
    return filtered_song, filtered_article

def clean_data(data):
    song_list = []
    unique_words_list = []
    count = 0
    for key, song_info in data['lyrics'].items():
        title, artist = key
        inner_title = song_info['title']
        if count%1000==0:
            print('%d: %s' %(count, inner_title))
        inner_artist = song_info['artist']
        song_lyrics = song_info['lyrics']
    
#       song_lyrics_norm = re.sub(r'[^a-zA-Z0-9-\']', ' ', song_lyrics).strip()
#       song_lyrics_split = song_lyrics_norm.lower().split()
        song_lyrics_norm, song_lyrics_split = remove_stop_and_punct(song_lyrics)
    
        if len(song_lyrics_split) <= MAX_SONG_LENGTH:       
            song_list.append(song_lyrics_norm)
            unique_words_list = list(set(unique_words_list + song_lyrics_split))
        count+=1
        
        if count >= SONG_PER_GENRE:
            print('hit max songs: %d' %(SONG_PER_GENRE))
            print('songs left out: %d' %(len(data['lyrics'])-SONG_PER_GENRE))
            return song_list, unique_words_list
       
    return song_list, unique_words_list
# initial data pre-processing
# assuming a list of tokenized data 
# vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(max_document_len)
print('cleaning data')
lyrics = []
lyrics_labels = []
unique_words_set = []
for data in pickle_lyrics:
    genre = data['genre']
    print('cleaning: %s' %(genre))
    song_list, unique_words = clean_data(data)
    unique_words_set = list(set(unique_words_set+unique_words))
    song_labels = [genre_index[genre]]*len(song_list)
    
    lyrics = lyrics + song_list
    lyrics_labels = lyrics_labels + song_labels
print('\n\n')
print('number of songs: %d' %(len(lyrics)))
print('number of lyrics: %d' %(len(lyrics_labels)))
print('number of unique words: %d' %(len(unique_words_set)))
print('finished cleaning data')

cleaning data
cleaning: Christian
0: Never Gonna Let Me Go
1000: By His Wounds
2000: Living For You
3000: Whole World
4000: I Manipulate
hit max songs: 4500
songs left out: 2395
cleaning: Pop
0: Emotional
1000: Day Is Done
2000: Roll Witchu
3000: Broken Glass
4000: Bitchin' Summer
hit max songs: 4500
songs left out: 1318
cleaning: Rock
0: Pleasure
1000: Interesting Drug
2000: Vilify
3000: Almost (Sweet Music)
4000: Unbelievers
hit max songs: 4500
songs left out: 1096
cleaning: Country
0: Here Comes My Baby
1000: Passionate Kisses
2000: My Favorite Memory
3000: Tattoos On This Town
4000: My Old Man
hit max songs: 4500
songs left out: 868
cleaning: Rap
0: Can't Tell
1000: Dollar Sign
2000: U Don't Know Me
3000: Barbie Dreams
4000: Haters Listen Up!
hit max songs: 4500
songs left out: 247



number of songs: 22500
number of lyrics: 22500
number of unique words: 54404
finished cleaning data


In [21]:
# MAX_UNIQUE_WORDS = len(unique_words_set)
MAX_UNIQUE_WORDS = 20000
# MAX_SONG_LENGTH = 1000

# data preparing
print('tokenizing')
tokenizer = keras.preprocessing.text.Tokenizer(num_words=MAX_UNIQUE_WORDS)
tokenizer.fit_on_texts(lyrics)
sequences = tokenizer.texts_to_sequences(lyrics)

word_index = tokenizer.word_index
print('Unique words tokens %d' % (len(word_index)))

data = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SONG_LENGTH)
labels = keras.utils.to_categorical(np.asarray(lyrics_labels))

print('finished tokenizing')


tokenizing
Unique words tokens 54391
finished tokenizing


In [22]:
print(data[0])
print(labels[0])

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

print(data[0])
print(labels[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [23]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]



t_data = data[:int(data.shape[0]*.1)]
t_labels = labels[:int(data.shape[0]*.1)]

# NO GPU so must downsize
CPU=True
if not CPU:
    num_test_samples = int(TEST_SPLIT * data.shape[0])
    num_validation_samples = int(VALIDATION_SPLIT * (data.shape[0]-num_test_samples))

    x_test = data[:num_test_samples]
    y_test = labels[:num_test_samples]
    x_val = data[num_test_samples:num_test_samples+num_validation_samples]
    y_val = labels[num_test_samples:num_test_samples+num_validation_samples]
    x_train = data[num_test_samples+num_validation_samples:]
    y_train = labels[num_test_samples+num_validation_samples:]
    
else:
    num_test_samples = int(TEST_SPLIT * t_data.shape[0])
    num_validation_samples = int(VALIDATION_SPLIT * (t_data.shape[0]-num_test_samples))

    x_test = t_data[:num_test_samples]
    y_test = t_labels[:num_test_samples]
    x_val = t_data[num_test_samples:num_test_samples+num_validation_samples]
    y_val = t_labels[num_test_samples:num_test_samples+num_validation_samples]
    x_train = t_data[num_test_samples+num_validation_samples:]
    y_train = t_labels[num_test_samples+num_validation_samples:]
    
print('data tensor:', data.shape)
print('test tensor:', x_test.shape)
print('validate tensor:', x_val.shape)
print('train tensor:', x_train.shape)
print('valid splits: ', x_test.shape[0]+x_val.shape[0]+x_train.shape[0] == data.shape[0])
print('label tensor:', labels.shape)
print('test tensor:', y_test.shape)
print('validate tensor:', y_val.shape)
print('train tensor:', y_train.shape)
print('valid splits: ', y_test.shape[0]+y_val.shape[0]+y_train.shape[0] == data.shape[0])

print('Preparing embedding matrix.')
# prepare embedding matrix
unique_words_count = min(MAX_UNIQUE_WORDS, len(word_index))
embedding_matrix = np.zeros((unique_words_count, EMBEDDING_DIM))

for word, i in word_index.items():
    if i >= MAX_UNIQUE_WORDS:
        continue
    embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        # potentially can improve if OOV words are handled differently        
        embedding_matrix[i] = embedding_vector
        
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = keras.layers.Embedding(unique_words_count,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SONG_LENGTH,
                            trainable=False)


data tensor: (22500, 1000)
test tensor: (450, 1000)
validate tensor: (360, 1000)
train tensor: (1440, 1000)
valid splits:  False
label tensor: (22500, 5)
test tensor: (450, 5)
validate tensor: (360, 5)
train tensor: (1440, 5)
valid splits:  False
Preparing embedding matrix.


In [24]:
# save model
def save_model(filename,weights_filename):
    # serialize model to JSON
    model_json = model.to_json()
    with open(filename, "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(weights_filename)
    print("Saved model to disk")


In [25]:

def load_model(filename,weights_filename):
    # load json and create model
    json_file = open(filename, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(weights_filename)
    print("Loaded model from disk")
    return loaded_model


In [26]:
print('Training model.')

sequence_input = tf.keras.layers.Input(shape=(MAX_SONG_LENGTH,))
embedded_sequences = embedding_layer(sequence_input)

#Model 1
l_cov1= tf.keras.layers.Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = tf.keras.layers.MaxPooling1D(5)(l_cov1)
l_drop1= tf.keras.layers.Dropout(dropout)(l_pool1)
l_cov2 = tf.keras.layers.Conv1D(128, 5, activation='relu')(l_drop1)
l_pool2 = tf.keras.layers.MaxPooling1D(5)(l_cov2)
l_drop2 = tf.keras.layers.Dropout(dropout)(l_pool2)
l_cov3 = tf.keras.layers.Conv1D(128, 5, activation='relu')(l_drop2)
l_pool3 = tf.keras.layers.MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = tf.keras.layers.Flatten()(l_pool3)
l_dense = tf.keras.layers.Dense(128, activation='relu')(l_flat)
preds = tf.keras.layers.Dense(len(genre_index), activation='softmax')(l_dense)

optimizer = tf.keras.optimizers.RMSprop(lr=learning_rate, clipnorm = max_grad_norm)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.summary()

Training model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 1000, 200)         4000000   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 996, 128)          128128    
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 199, 128)          0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 39, 128)           0    

In [27]:
model_details = model.fit(x_train, y_train,
            epochs=100,
            shuffle=True,
            verbose=1,
            validation_data=(x_val, y_val))

scores = model.evaluate(x_test,y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Train on 1440 samples, validate on 360 samples
Epoch 1/100
1440/1440 [==============================] - 39s 27ms/sample - loss: 1.5736 - acc: 0.3111 - val_loss: 1.5357 - val_acc: 0.2444
Epoch 2/100
1440/1440 [==============================] - 36s 25ms/sample - loss: 1.3733 - acc: 0.3722 - val_loss: 1.4459 - val_acc: 0.4194
Epoch 3/100
1440/1440 [==============================] - 36s 25ms/sample - loss: 1.2523 - acc: 0.4701 - val_loss: 1.2949 - val_acc: 0.4361
Epoch 4/100
1440/1440 [==============================] - 39s 27ms/sample - loss: 1.1539 - acc: 0.5146 - val_loss: 1.2403 - val_acc: 0.4833
Epoch 5/100
1440/1440 [==============================] - 43s 30ms/sample - loss: 1.0725 - acc: 0.5333 - val_loss: 1.2054 - val_acc: 0.4778
Epoch 6/100
1440/1440 [==============================] - 37s 26ms/sample - loss: 1.0210 - acc: 0.5611 - val_loss: 1.2090 - val_acc: 0.4806
Epoch 7/100
1440/1440 [==============================] - 37s 26ms/sample - loss: 0.9482 - acc: 0.6146 - val_loss: 1.121

KeyboardInterrupt: 

In [ ]:
save_model(MODEL_SAVE_FILE, MODEL_SAVE_WEIGHTS_FILE)